# NmapBuddy - Automated Port & Vulnerability Scanner


## 📚 Table of Contents
1. [Overview](#overview)
2. [Setup](#setup)
3. [Timestamp + Logging](#timestamp)
4. [Target Selection](#target)
5. [Nmap Scan Execution](#scan)
6. [Output & Results](#results)
7. [Future Features](#future)


# NmapBuddy - Automated Port and Vulnerability Scanner
---
**Created by:** Tamieka Horne  
**Date Started:** 4/27/2025  

This project automates port scanning and basic vulnerability detection using Nmap and Python.  
It is designed to streamline manual security assessments by allowing users to quickly identify open ports and potential vulnerabilities on a target system or network.


## 🛠️ Tools & Technologies

This project was built using:
- **Python 3.x**
- **Nmap** – for performing port and vulnerability scans
- **subprocess module** – to execute shell commands from within Python
- **datetime module** – for timestamping logs
- **Jupyter Notebook** – for interactive scripting, documentation, and analysis

These tools were chosen for their accessibility, effectiveness, and ability to support real-time iteration and automation in security assessments.


In [1]:
# Install the python-nmap package
%pip install python-nmap

import nmap

# Initialize the scanner
scanner = nmap.PortScanner()

# Define target and ports
target = 'scanme.nmap.org'
ports = '20-100'

# Run a basic scan
print(f"Scanning {target} for ports {ports}...")

scanner.scan(target, ports)

# Display the scan results
for host in scanner.all_hosts():
    print(f"\nHost: {host} ({scanner[host].hostname()})")
    print(f"State: {scanner[host].state()}")

    for protocol in scanner[host].all_protocols():
        print(f"\nProtocol: {protocol}")
        ports = scanner[host][protocol].keys()
        for port in ports:
            print(f"Port: {port}\tState: {scanner[host][protocol][port]['state']}")


  Preparing metadata (setup.py) ... done
  Created wheel for python-nmap: filename=python_nmap-0.7.1-py2.py3-none-any.whl size=20633 sha256=1e08ee243e0da8594193da955d8c4d1f9dacd39645ea8f9fe009505fe80a0fcd
  Stored in directory: /Users/tamiekahorne/Library/Caches/pip/wheels/06/fc/d4/0957e1d9942e696188208772ea0abf909fe6eb3d9dff6e5a9e
Successfully built python-nmap
Note: you may need to restart the kernel to use updated packages.
Scanning scanme.nmap.org for ports 20-100...

Host: 45.33.32.156 (scanme.nmap.org)
State: up

Protocol: tcp
Port: 22	State: open
Port: 25	State: filtered
Port: 53	State: open
Port: 80	State: open


## Basic Nmap Scan
This cell runs a simple Nmap port scan on `scanme.nmap.org` from ports 20 to 100.
It prints the open ports and the status of the target host.


In [12]:
import nmap
import json
from datetime import datetime
from colorama import Fore, Style, init

init(autoreset=True)

def log_scan(target, format_type):
    with open("scan_log.txt", "a") as log:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log.write(f"[{timestamp}] Scanned {target} | Format: {format_type}\n")

def save_as_text(scan_data, host_ip, original_target):
    filename = f"{original_target.replace('.', '_')}_scan.txt"
    with open(filename, "w") as f:
        f.write(f"Host: {host_ip} ({original_target})\n")
        f.write(f"State: {scan_data['scan'][host_ip]['status']['state']}\n\n")
        f.write("Protocol: tcp\n")
        for port, details in scan_data['scan'][host_ip]['tcp'].items():
            f.write(f"Port: {port}\tState: {details['state']}\n")
    print(Fore.GREEN + f"Scan results saved to {filename}")
    log_scan(original_target, "Text")

def save_as_json(scan_data, host_ip, original_target):
    filename = f"{original_target.replace('.', '_')}_scan.json"
    with open(filename, "w") as f:
        json.dump({"scan": {host_ip: scan_data[host_ip]}}, f, indent=2)
    print(Fore.GREEN + f"Scan results saved to {filename}")
    log_scan(original_target, "JSON")


def run_scan(target):
    print(Fore.CYAN + f"\nScanning {target} for ports 20-100...")
    nm = nmap.PortScanner()
    try:
        nm.scan(hosts=target, arguments='-p 20-100 -sV')
        scan_data = nm.all_hosts()

        if not scan_data:
            print(Fore.RED + f"No hosts found for {target}.")
            return None, None

        resolved_ip = scan_data[0]  # like "45.33.32.156"
        return nm, resolved_ip

    except Exception as e:
        print(Fore.RED + f"Scan error for {target}: {e}")
        return None, None

def menu():
    print(Style.BRIGHT + "===== nmapBuddy (Multi-Target Edition) =====")
    targets_input = input(Fore.BLUE + "Enter target(s), separated by commas: ").strip()
    raw_targets = [t.strip() for t in targets_input.split(",") if t.strip()]

    if not raw_targets:
        print(Fore.RED + "No valid targets entered. Exiting.")
        return

    for raw_target in raw_targets:
        if raw_target.isdigit():
            print(Fore.RED + f"Skipping invalid target: {raw_target}")
            continue

        scanner, host_ip = run_scan(raw_target)
        if not scanner or not host_ip:
            continue

        print(Fore.YELLOW + f"\nChoose output format for {raw_target}:")
        print(Fore.BLUE + "1. Save as text")
        print(Fore.BLUE + "2. Save as JSON")

        choice = input(Fore.BLUE + "Enter your choice (1 or 2): ").strip()

        if choice == "1":
            save_as_text(scanner, host_ip, raw_target)
        elif choice == "2":
            save_as_json(scanner, host_ip, raw_target)
        else:
            print(Fore.RED + "Invalid choice. Skipping output.\n")

menu()


===== nmapBuddy (Multi-Target Edition) =====

Scanning scanme.nmap.org for ports 20-100...

Choose output format for scanme.nmap.org:
1. Save as text
2. Save as JSON
Scan results saved to scanme_nmap_org_scan.json

Scanning 127.0.0.1 for ports 20-100...

Choose output format for 127.0.0.1:
1. Save as text
2. Save as JSON
Scan results saved to 127_0_0_1_scan.json


In [13]:
import os

# Create output folders
os.makedirs("output/json", exist_ok=True)
os.makedirs("output/txt", exist_ok=True)
os.makedirs("images", exist_ok=True)
